In [1]:
# Collecting tweets from zenodo twitter dataset

# Download tweepy to use Twitter API
!pip3 install tweepy
import tweepy

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python37\python.exe -m pip install --upgrade pip' command.


In [2]:
# Open file with Twitter developer keys
keys_file =open("TwitterAPI_Keys_Secret.txt", "r")
keys = keys_file.readlines()

# Twitter developer keys
consumer_key = str(keys[0])
consumer_secret = str(keys[1])
access_token = str(keys[2]) 
access_token_secret = str(keys[3])

consumer_key = consumer_key[:len(consumer_key)-1]
consumer_secret = consumer_secret[:len(consumer_secret)-1]
access_token = access_token[:len(access_token)-1]
access_token_secret = access_token_secret[:len(access_token_secret)-1]


In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

In [4]:
import csv

tsv_file = "full_dataset-clean.tsv"
data = open(tsv_file)
read_tsv = csv.reader(data, delimiter="\t")

row_count = 1
tweet_ids = []
for row in read_tsv:
    #if row_count <= 2:
    #    row_count += 1
    #    continue
    tweet_ids.append(row[0])
    #print(row_count, row[0])
    row_count += 1
    if row_count % 1000000 == 0:
        print(row_count)
print(len(tweet_ids))

1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
22000000
23000000
24000000
25000000
26000000
27000000
28000000
29000000
30000000
30990646


In [13]:
# Collect 5000 random tweets from each month Jan-Apr

import csv

tsv_file = "full_dataset-clean.tsv"
data = open(tsv_file)
read_tsv = csv.reader(data, delimiter="\t")

jan_tweets = []
feb_tweets = []
mar_tweets = []
apr_tweets = []

row_count = 1
for row in read_tsv:
    date = row[1]
    if date[5:7] == "01":
        jan_tweets.append(row[0])
    if date[5:7] == "02":
        feb_tweets.append(row[0])
    if date[5:7] == "03":
        mar_tweets.append(row[0])
    if date[5:7] == "04":
        apr_tweets.append(row[0])
    row_count+=1
print(len(jan_tweets), len(feb_tweets), len(mar_tweets), len(apr_tweets))


1329579 4745851 21049512 3865703


In [18]:
import random
from random import sample

jan_sample = sample(jan_tweets, 9000)
feb_sample = sample(feb_tweets, 9000)
mar_sample = sample(mar_tweets, 9000) 
apr_sample = sample(apr_tweets, 9000) 

jan_file = open("jan_tweet_sample.txt", "w")
feb_file = open("feb_tweet_sample.txt", "w")
mar_file = open("mar_tweet_sample.txt", "w")
apr_file = open("apr_tweet_sample.txt", "w")

for tweetID in jan_sample:
    jan_file.write(tweetID + "\n")
for tweetID in feb_sample:
    feb_file.write(tweetID + "\n")
for tweetID in mar_sample:
    mar_file.write(tweetID + "\n")
for tweetID in apr_sample:
    apr_file.write(tweetID + "\n")

In [24]:
print(len(jan_sample))
print(jan_sample[8993:9000])
print(feb_sample[8993:9000])
print(mar_sample[8993:9000])
print(apr_sample[8993:9000])

9000
['1222303294770466822', '1222900207072899079', '1222233956772347904', '1221864980565450754', '1223017419280654336', '1223143837289406464', '1221986758830821376']
['1224862199803449345', '1232770948471906305', '1231702296393728000', '1229871972844220416', '1230787564765925376', '1229441507670872070', '1224829567648419840']
['1240731532592996352', '1239464396579057664', '1239649817414025223', '1238665997575270400', '1239734320174190594', '1240915792910147584', '1238496492807229440']
['1245223506469675008', '1245653750338875392', '1245499868967124992', '1246608235076993026', '1245382034052149249', '1245145122674786306', '1245433800743673857']


In [32]:
import re
!pip3 install unidecode
from unidecode import unidecode
#!pip3 install google_trans_new
#import google_trans_new
#from google_trans_new import google_translator
import csv

#read_csv('path_to_file', sep='\t')

mar_file = open("mar_tweet_sample.txt", "r")
tweet_ids = []
for row in mar_file:
    tweet_ids.append(row)
print(len(tweet_ids))
print(tweet_ids[0], tweet_ids[1])

tweet_count = 0

with open('mar_tweets.tsv', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(['tweetID', 'tweetTime', 'tweetText', 'tweetLanguage', 'tweetCoordinates', 'tweetPlace'])

    for i in range(6320, len(tweet_ids)):
        if tweet_count == 500:
            print("500 tweets")
            break
        try:
            # get full tweet text from tweet id
            tweet = api.get_status(tweet_ids[i], tweet_mode="extended")
            #print(tweet.full_text + "\n")

            # remove links beginning with http
            text = re.sub(r"http\S+", "", tweet.full_text)
            #print(text + "\n")

            # remove emojis
            print(i, unidecode(text), '\n')
            #detector = google_translator()
            #lang = detector.detect(text)[1]

            time_created = tweet.created_at
            if tweet.coordinates != None:
                coordinates = tweet.coordinates
            else: coordinates = ''
            if tweet.place != None:
                place = tweet.place
            else: place = ''

            tsv_writer.writerow([tweet_ids[i], time_created, unidecode(text), tweet.lang, coordinates, place])
            tweet_count += 1

        except tweepy.TweepError:
            print("tweet", i, "not found")
            #tsv_writer.writerow([tweet_ids[i], '', '', '', '', ''])
        except UnicodeEncodeError:
            print("tweet", i, "unicode error")
            print("tweet count", tweet_count)
            tsv_writer.writerow([tweet_ids[i], time_created, text.encode("utf-8"), tweet.lang, coordinates, place])
        if tweet_count % 1000 == 0:
            print("current", tweet_count)

You should consider upgrading via the 'c:\program files\python37\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable
9000
1243821363485343750
 1235210106326876161

tweet 6320 not found
current 0
tweet 6321 not found
current 0
tweet 6322 not found
current 0
6323 Reasonable and safe.  

6324 Coronavirus: Policlinico, contagiato anche un infettivologo - Il Giorno  #COVID19 #coronavirus 

6325 CoronaVirus, altri 19 nuovi casi positivi in Toscana -   

6326  #Pandemic a time of understanding value of brotherhood, cardinal says  

6327 vsy j'avais grave de l'espoir, au final mon controle d'svt j'v devoir le faire... ptn le coronavirus il sert a rien jsuis deg 

6328 @PoliticsReid The lawsuits from the future victims of these negligent decisions will destroy the economic interest that this person with questionable morality is trying so hard to protect. @JerryFalwellJr lacks the values required from real leadership. #COVID19 

6329 Got any stock tips?  

tweet 6330 not found
tweet 6331 not found
6332 @realDonaldTrump Hey everyone. 

6392 Free those Sigs  Definitely snapped with the video    

tweet 6393 not found
6394 @sumitkashyapjha might be they are not missing. Instead they are working hard in their offices to tackle the upcoming situations due to corona virus along with maintaining social distance, so that they don't get infected. 
Common, r u so bored to create these stories?  

6395 #CuarentenaNacional #COVID19 Encuentra aqui  los detalles de las universidades en Mexico que ofrecen cursos online gratuitos para hacer esta cuarentena  

6396 Un papa eretico?  

6397 Famosos critican realizacion del Vive Latino pese a coronavirus  via @El_Universal_Mx 

tweet 6398 not found
6399 @dabitch 

Come to Western Aust.. Get Kung Flu, get to be quarantined with small cute animals. 

 

6400 @DrManaouda Le Coronavirus de ca... Pardon excusez nous Mr le ministre 

6401 Aquesta merda segueix fent historia del mal periodisme: LA FOTO ERA D'UN ALTRE DIA!!!  

6402 Qui da me   

6403 En Liverpool les da igual el coronavirus.

tweet 6458 not found
tweet 6459 not found
6460 @Allison_Burnett @NotBroCoach My neighbor took ill..after pneumonia  vaccine..got it..and passed 

6461 Trump Calls Out Fake News Media for False Reporting on Coronavirus Website   

6462 Gobierno de Italia no descarta un "bloqueo general" ante el avance del coronavirus #Emol  via @emol 

tweet 6463 not found
6464 Que casualidad que en plena crisis del #Coronavirus, donde segun el regimen "iba a garantizar las telecomunicaciones"... El satelite "Simon Bolivar" (que nunca se supo si funciono o en que ayudo) se salio de orbita .

?Y el satelite "Miranda" mas o menos? 

6465 @FuturFestival what's the crack with coronavirus then? Is the festival gonna get cancelled orrrr 

tweet 6466 not found
6467 @casilatino73 Y ...la realidad del covid19 :D 

6468 En language de film d'horreur :

"Ca va, c'est pas comme si j'allais mourir !
Au fait, j'ai perdu ma virginite, il me reste 2 jours avant la retraite et voila une photo de ma famille.

Ah, et je s

6538 Emergenza #coronavirus, da oggi nuove disposizioni a Rivalta   

6539 HARUSNYA KEMAREN PAS PESTA NIKAHAN MANTAN AING DIDATENGIN POLISI BUAT DIBUBARIN GINI NIH!  :(  

6540 Coronavirus Lockdown May Save More Lives By Preventing Pollution Than By Preventing Infection  

6541 Coronavirus update: US-returned doctor allegedly found treating patients; ignored quarantine rules - india news   

6542 [?]  

6543 I'm afraid that this #COVID19 pandemic will last longer than we expected. [?] I mean, it will not be considered as "pandemic" for nothing. 
Past pandemics lasted at least a year.  

tweet 6544 not found
6545 @jokowi @prabowo 
Yth.bapak berdua.
Tolong ambil alih semua stasiun TV dan siarkan saja hal yg berhubungan dengan sebab &amp; akibat tertular Covid-19 (24/7)
Please hanya bapak berdua yg bisa melakukan ini...
Kalau tidak bakal jatuh korban lebih banyak lagi..!!!! 

tweet 6546 not found
6547 .________________________________________.  

6548 @ColetaFuriosa @24h_tve Perdona, pero

6607 On this day in 2012 this cricketing great @sachin_rt played his last ODI but he is still batting for #India on a different pitch, making other aware to be Safe from #CoronaVirus  

6608 Suspected case of coronavirus detected in Paga   

6609 Lo sport al tempo del coronavirus: ecco le regole dei medici sportivi   

6610 #Kazakhstan: The government is implementing enhanced measures to reduce the spread of #COVID19.  Travelers should be prepared for travel restrictions to be put into effect with no notice. See alert for an overview of the current quarantine policy.    

6611 Por que brotes como el del coronavirus crecen exponencialmente y como 'aplanar la curva'  

tweet 6612 not found
6613 First deaths by COVID-19 reported in Myanmar: 1 people have died out of 14 confirmed cases.

A 7.14% fatality rate. 

6614 Good news for Nkurunziza: Elections are going to be postponed following the #COVID19 outbreak. 

6615 coronavirus acabe  

tweet 6616 not found
tweet 6617 not found
6618 @Germ

6676 OMS felicita a Mexico por su actuacion ante el coronavirus #COVID19 #Coronavirusmexico   

6677 Edition Besancon | Coronavirus : 11 morts dans un Ehpad du Doubs -  

6678 Coronavirus: Radio listening booms while music streaming stalls   

6679 Is locked-down SA a frog in the warming Covid-19 pot?  

6680 "How can HR leaders manage the #TalentChallenges of #COVID19?" 

Did you miss our webinar yesterday? Watch the recording now! 

Get hands-on advice on how to keep your #TalentStrategy on track in the light of #CoronavirusOutbreak here:

  

6681 Coronavirus: Australian newspaper prints extra pages to help out in toilet paper shortage  

6682 The most cruel joke ever played on PMC Bank depositors. Due to the pandemic Banks are delivering cash to depositors homes. What are PMC depositors getting? Over 20 dead and no help. Should the rest be ordering for death from the bank? Why this cruelty
 

6683 Eronkhii said: Bid ene baidlaar neg zhiliig davaad garch chadakhaar baina. Tsagiin ba

6756 Repost : @DishubDKI_JKT .
Dalam upaya mencegah risiko penyebaran virus COVID-19 di Jakarta, semua BUMD transportasi umum @mrtjakarta, pt_transjakarta dan @lrtjkt menyediakan hand sanitizer atau pembersih tangan yang...  

6757 Washington state residents frustrated over obstacles to get coronavirus tests  

tweet 6758 not found
6759 Wow markets have run out of sanitizers #coronavirus 

6760 Se han parado a pensar que la mayoria de las fabricas de Amancio Ortega se encuentran en Asia donde muchos paises estan luchando por el coronavirus y donde muchas personas tendra que seguir trabajando en lugar de quedarse en cuarentena para evitar mas contagios. #DonAmancio 

6761 No le importa el #coronavirus 
Lo dijo en 2016: !dadme el CNI y dadme los telediarios!
Sabe lo q hace 
#CuarentenaTotal  

6762 I've got a sneeze, was worried that it could be Coronavirus then realised its MARCH already so probably hayfever 

6763 L'ignorance tue... cette dame qui essayait de me convaincre toute a l'he

6823 @JaggerMickOZ Absoluitely. He can give her Coronavirus ! 

tweet 6824 not found
tweet 6825 not found
6826 @DianaSalazarM2 Tu eres lo peor que le paso al pais y lo sabes, ya saldra a la luz tus negocios con Molina, empieza a buscar un pais para esconderte por que el coronavirus es poco para lo que tendras que pagar por todas las injusticias cometidas empleadita domestica 

6827 We all Indians are nearly 137 crores. I want my Dear friends, brothers and sisters, keep your hands with the Prime Minister as much as possible, this is a serious situation for all of us.
Keep retweet all. 

We will win, India will win.
We will win, CoronaVirus will lose.   

6828 21:45 Umraniye Cengiz Topel tesislerinde halisaha macinda oynacak bir kisi araniyor #coronavirusturkey #coronatuerkiye #COVID19 #Korona #CoronaVirusChallenge 

tweet 6829 not found
6830 To everyone: Please be safe during this coronavirus incident. Just do common sense practices to encourage low-risk of getting it. Be healthy and we

6915 I'm #prochoice, but I agree. Bill padding has to #STOP!  

6916 They're heroes for sure :')  

6917 GDC 2020 "indefinitely postponed" after losing Microsoft, Sony, Epic to coronavirus: GDC 2020 finally succumbs to coronavirus after a number of big exhibitors pull out, including Sony, Microsoft, Epic and Oculus.   

tweet 6918 not found
tweet 6919 not found
6920 What does the coronavirus mean for the U.S. health care system? Some simple math offers alarming...  via @statnews #actnow #coronavirus 

6921 @sparksthetoby  

tweet 6922 not found
tweet 6923 not found
6924 Report: Coronavirus has been spreading for weeks in the US undetected  

6925 Despicable, Deplorable, Desperate......  

tweet 6926 not found
6927 ...,which is EXACTLY why we're wondering if @idriselba claim of "immunity" was TOO SOON (it most likely WAS/IS) (March 27, 2020; 13:48 HKT) #covid19 

6928 Except he could still do that and not be running for president simultaneously. The two aren't linked.  

6929 Minsal ges

# Sentiment Analysis Using BERT

In [ ]:
!pip3 install transformers[torch]
#!pip3 install git+https://github.com/huggingface/transformers
import transformers
from transformers import BertModel, AdamW, get_linear_schedule_with_warmup, BertTokenizer
#!pip3 install torch
import torch

In [12]:
import numpy as np
!pip3 install pandas
import pandas as pd
!pip3 install seaborn
import seaborn as sns
!pip3 install pylab
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
!pip3 install sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
!pip3 install ipywidgets

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python37\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python37\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement pylab (from versions: none)
ERROR: No matching distribution found for pylab
You should consider upgrading via the 'c:\program files\python37\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable
Using legacy setup.py install for sklearn, since package 'wheel' is not installed.
    Running setup.py install for sklearn: started
    Running setup.py install for sklearn: finished with status 'done'


You should consider upgrading via the 'c:\program files\python37\python.exe -m pip install --upgrade pip' command.


In [4]:
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python37\python.exe -m pip install --upgrade pip' command.


In [ ]:
df = pd.read_csv("tweets - Copy 1-6602.tsv", sep='\t')
df.head()
